In [1]:
import pandas as pd
import numpy as np
import mysql.connector

import random
import datetime
import time

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.options.display.max_columns = None

# 1 Read data and preprocess

## 1.1 Read and preprocess listing data

In [3]:
listing = pd.read_csv('listings.csv', encoding='utf-8')

In [4]:
cols_to_drop = [
    'scrape_id',
    'last_scraped', 
    'host_total_listings_count', 
    'bathrooms', 
    'calendar_updated', 
    'calendar_last_scraped', 
    'license',
    'neighbourhood_group_cleansed'
]

In [5]:
listing.drop(cols_to_drop, axis = 1, inplace = True)

In [6]:
### drop na for some columns
dropna_cols = [
    'name',
    'host_name',
    'host_since',
    'host_location',
    'host_is_superhost',
    'host_thumbnail_url',
    'host_picture_url',
    'host_listings_count',
    'host_has_profile_pic',
    'minimum_minimum_nights',
    'maximum_minimum_nights',
    'minimum_maximum_nights',
    'maximum_maximum_nights',
    'minimum_nights_avg_ntm',
    'maximum_nights_avg_ntm'
]
listing.dropna(subset = dropna_cols, inplace=True)

In [7]:
# some column has nan stored as string
listing = listing[listing.host_name != 'Nan']

In [8]:
fillna_values = {
    'bathrooms_text': '0 bath',
    'bedrooms' : 0,
    'beds': 0,
}

In [9]:
listing.fillna(fillna_values, inplace=True)

In [10]:
listing = listing[(listing.host_acceptance_rate.notnull()) | (listing.first_review.isnull())]

In [11]:
listing['price'] = listing.price.apply(lambda x: x.replace(',', '').replace('$', ''))

In [12]:
listing_type_dict = {
    'host_is_superhost': bool,
    'host_has_profile_pic': bool,
    'host_identity_verified': bool,
    'has_availability': bool,
    'instant_bookable': bool,
}

In [13]:
for col in listing_type_dict.keys():
    listing[col] = listing[col].apply(lambda x: True if x == 't' else False)

## 1.2 Read and preprocess reviews data

In [14]:
reviews = pd.read_csv('reviews.csv', encoding='utf-8')

In [15]:
reviews = reviews.dropna()

## 1.3 Read and proprocess calendar data

In [16]:
# calendar = pd.read_csv('calendar.csv', encoding='utf-8')

In [17]:
# calendar['price'] = calendar.price.apply(lambda x: x.replace(',', '').replace('$', ''))
# calendar['adjusted_price'] = calendar.adjusted_price.apply(lambda x: x.replace(',', '').replace('$', ''))

In [18]:
# calendar_type_dict = {
#     'available': bool,
# }

In [19]:
# calendar = calendar.astype(calendar_type_dict)

# 2 Transfer Pandas dataframe to numpy to insert into mysql

In [20]:
host_table = listing.loc[:, ['host_id',
                             'host_url',
                             'host_name',
                             'host_since',
                             'host_location',
                             'host_about',
                             'host_response_time',
                             'host_response_rate',
                             'host_acceptance_rate',
                             'host_is_superhost',
                             'host_thumbnail_url',
                             'host_picture_url',
                             'host_neighbourhood',
                             'host_listings_count',
                             'host_verifications',
                             'host_has_profile_pic',
                             'host_identity_verified']].drop_duplicates().to_numpy()
host_table = [tuple(i) for i in host_table]
# host_table = [tuple(i) for i in host_table if (i[5] is np.nan or len(i[5]) <= 1000)]

In [21]:
listing_table = listing.loc[:, ['id', 'listing_url', 'name', 'description', 'neighborhood_overview', 
                                'picture_url', 'host_id', 'neighbourhood', 'neighbourhood_cleansed', 
                               'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 
                               'bathrooms_text', 'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights',
                               'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 
                               'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 
                               'has_availability', 'availability_30', 'availability_60', 'availability_90',
                               'availability_365', 'number_of_reviews', 'number_of_reviews_ltm',
                               'number_of_reviews_l30d', 'first_review', 'last_review', 'review_scores_rating',
                                'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 
                                'review_scores_communication', 'review_scores_location', 'review_scores_value',
                                'instant_bookable', 'reviews_per_month', 'calculated_host_listings_count', 
                                'calculated_host_listings_count_entire_homes', 
                                'calculated_host_listings_count_private_rooms', 
                                'calculated_host_listings_count_shared_rooms']].drop_duplicates().to_numpy()
listing_table = [tuple(i) for i in listing_table]

In [22]:
reviewer_table = reviews.loc[:, ['reviewer_id', 'reviewer_name']].drop_duplicates().to_numpy()
reviewer_table = [tuple(i) for i in reviewer_table]

In [23]:
review_table = reviews.loc[:, ['id', 'listing_id', 'reviewer_id', 'comments', 'date']].drop_duplicates().to_numpy()
review_table = [tuple(i) for i in review_table if (i[1] in listing.id.values and i[0] != 35507141)]

In [24]:
# calendar_table = calendar.drop_duplicates().to_numpy()
# calendar_table = [tuple(i) for i in calendar_table if i[0] in listing.id.values]

# 3 Connect Mysql

In [25]:
db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database='project'
)

In [26]:
cursor = db.cursor()

# 4 Create table in mysql database

In [27]:
# drop table if already exist in database
query = '''
DROP TABLE IF EXISTS transaction, calendar, review, reviewer, listing, host CASCADE
'''
cursor.execute(query)

In [28]:
query = '''CREATE TABLE host(
    host_id BIGINT,
    host_url VARCHAR(5000),
    host_name VARCHAR(100),
    host_since DATE,
    host_location VARCHAR(500),
    host_about VARCHAR(6000),
    host_response_time VARCHAR(50),
    host_response_rate VARCHAR(50),
    host_acceptance_rate VARCHAR(50),
    host_is_superhost BOOLEAN,
    host_thumbnail_url VARCHAR(500),
    host_picture_url VARCHAR(500),
    host_neighbourhood VARCHAR(50),
    host_listings_count INT,
    host_verifications VARCHAR(500),
    host_has_profile_pic BOOLEAN,
    host_identity_verified BOOLEAN,
    PRIMARY KEY(host_id)
)'''
cursor.execute(query)

In [29]:
query = f'''
    INSERT INTO host VALUES ({', '.join(['%s'] * 17)})
'''
cursor.executemany(query, host_table)
del host_table

In [30]:
query = '''
CREATE TABLE listing(
    listing_id BIGINT,
    listing_url VARCHAR(100),
    name VARCHAR(500),
    description VARCHAR(2000),
    neighborhood_overview VARCHAR(2000),
    picture_url VARCHAR(500),
    host_id BIGINT,
    neighbourhood VARCHAR(100),
    neighbourhood_cleansed VARCHAR(100),
    latitude DECIMAL(25,18),
    longitude DECIMAL(25, 18),
    property_type VARCHAR(100),
    room_type VARCHAR(100),
    accommodates INT,
    bathrooms_text VARCHAR(100),
    beds INT,
    amenities VARCHAR(2000),
    price DECIMAL(10, 2),
    minimum_nights INT,
    maximum_nights INT,
    minimum_minimum_nights INT,
    maximum_minimum_nights INT,
    minimum_maximum_nights INT,
    maximum_maximum_nights INT,
    minimum_nights_avg_ntm DECIMAL(16, 5),
    maximum_nights_avg_ntm DECIMAL(16, 5),
    has_availability BOOLEAN,
    availability_30 INT,
    availability_60 INT,
    availability_90 INT,
    availability_365 INT,
    number_of_reviews INT,
    number_of_reviews_ltm INT,
    number_of_reviews_l30d INT,
    first_review DATE,
    last_review DATE,
    review_scores_rating DECIMAL(10, 5),
    review_scores_accuracy DECIMAL(10, 5),
    review_scores_cleanliness DECIMAL(10, 5),
    review_scores_checkin DECIMAL(10, 5),
    review_scores_communication DECIMAL(10, 5),
    review_scores_location DECIMAL(10, 5),
    review_scores_value DECIMAL(10, 5),
    instant_bookable BOOLEAN,
    reviews_per_month DECIMAL(10, 5),
    calculated_host_listings_count INT,
    calculated_host_listings_count_entire_homes INT,
    calculated_host_listings_count_private_rooms INT,
    calculated_host_listings_count_shared_rooms INT,
    PRIMARY KEY(listing_id),
    FOREIGN KEY(host_id) REFERENCES host(host_id)
)
'''
cursor.execute(query)

In [31]:
query = f'''
    INSERT INTO listing VALUES ({', '.join(['%s'] * 49)})
'''
cursor.executemany(query, listing_table)
del listing_table

In [32]:
query = '''
CREATE TABLE reviewer (
    reviewer_id BIGINT,
    reviewer_name VARCHAR(50),
    PRIMARY KEY (reviewer_id)
)
'''
cursor.execute(query)

In [33]:
query = f'''
    INSERT INTO reviewer VALUES ({', '.join(['%s'] * 2)})
'''
cursor.executemany(query, reviewer_table)
del reviewer_table

In [34]:
query = '''
CREATE TABLE review (
    id BIGINT,
    listing_id BIGINT,
    reviewer_id BIGINT,
    comments VARCHAR(6000),
    date DATE,
    PRIMARY KEY (id),
    FOREIGN KEY (reviewer_id) REFERENCES reviewer(reviewer_id),
    FOREIGN KEY (listing_id) REFERENCES listing(listing_id)
)
'''
cursor.execute(query)

In [35]:
query = f'''
    INSERT INTO review VALUES ({', '.join(['%s'] * 5)})
'''
cursor.executemany(query, review_table)

In [36]:
query = '''
CREATE TABLE calendar (
    listing_id BIGINT,
    date DATE,
    available BOOLEAN,
    price DECIMAL(16,5),
    adjusted_price DECIMAL(16,5),
    minimum_nights INT,
    maximum_nights INT,
    FOREIGN KEY (listing_id) REFERENCES listing(listing_id)
)
'''
cursor.execute(query)

In [37]:
calendar_df = pd.read_csv('calendar.csv', encoding='utf-8', chunksize=4096 * 8)

In [38]:
query = f'''
    INSERT INTO calendar VALUES ({', '.join(['%s'] * 7)})
'''
count = 0
for i in calendar_df:
    if count % 10 == 0:
        print(count)
    count += 1
    i['price'] = i.price.apply(lambda x: x.replace(',', '').replace('$', ''))
    i['adjusted_price'] = i.adjusted_price.apply(lambda x: x.replace(',', '').replace('$', ''))
    i['available'] = i['available'].apply(lambda x: True if x == 't' else False)
    calendar_table = i.drop_duplicates().to_numpy()
    calendar_table = [tuple(i) for i in calendar_table if i[0] in listing.id.values]
    cursor.executemany(query, calendar_table)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190


In [39]:
# query = f'''
#     INSERT INTO calendar VALUES ({', '.join(['%s'] * 7)})
# '''
# cursor.executemany(query, calendar_table)
# del calendar_table

In [40]:
query = '''
CREATE TABLE transaction (
    transaction_id BIGINT,
    reviewer_id BIGINT,
    listing_id BIGINT,
    transcation_time DATE,
    check_in_date DATE,
    check_out_date DATE,
    actual_check_in_date DATE,
    total_cost DECIMAL(16, 5),
    Num_days_stayed INT,
    PRIMARY KEY (transaction_id),
    FOREIGN KEY (reviewer_id) REFERENCES reviewer(reviewer_id),
    FOREIGN KEY (listing_id) REFERENCES listing(listing_id)
)
'''
cursor.execute(query)

In [41]:
selected_reviews = random.choices(review_table, k = 1000)
    

In [42]:
for i, review in enumerate(selected_reviews):
    reviewer_id = review[2]
    listing_id = review[1]
    review_date = review[4]
    
    check_in = np.random.randint(low = 4, high = 20)
    actual_check_in = np.random.randint(low = 0, high = 2)
    check_out = np.random.randint(low = 0, high = check_in - actual_check_in)
    transaction_ndays = np.random.randint(low = 3, high = 100)
    
    date_format = '%Y-%m-%d'
    
    check_in_date = datetime.datetime.strptime(review_date, date_format) - datetime.timedelta(days=check_in)
    actual_check_in_date = check_in_date + datetime.timedelta(days=actual_check_in)
    check_out_date = datetime.datetime.strptime(review_date, date_format) - datetime.timedelta(days = check_out)
    transaction_date = check_in_date - datetime.timedelta(days = transaction_ndays)
    
    num_days_stayed = (check_out_date - check_in_date).days
    
    check_in_date = datetime.datetime.strftime(check_in_date, date_format)
    actual_check_in_date = datetime.datetime.strftime(actual_check_in_date, date_format)
    check_out_date = datetime.datetime.strftime(check_out_date, date_format)
    transaction_date = datetime.datetime.strftime(transaction_date, date_format)
    
    total_cost = random.uniform(50.0, 5000.0)

    data = (i, reviewer_id, listing_id, transaction_date, check_in_date, check_out_date,
            actual_check_in_date, total_cost, num_days_stayed)
    cursor.execute(f'INSERT INTO transaction VALUES {data}')

In [43]:
del review_table

In [44]:
db.commit()